In [1]:
from gallery_probes_generator import GalleryProbesGenerator
from eigenface_generator import EigenfaceGenerator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
norms = { "L1":lambda x: np.sum(np.abs(x)),
	  "L2":lambda x: np.sum(x**2),
	  "inf":lambda x: np.max(np.abs(x))
	}

def compute_distances_individual(data, query, norm="L2"):
	""" Compute distances.

	Computes the distances between the vectors (rows) of a dataset and a
	single query). Three distances are supported:
	  * Manhattan distance ("L1");
	  * squared Euclidean distance ("L2");
	  * Chebyshev distance ("inf").

	:param data: Dataset matrix with samples as rows.
	:param query: Query vector
	:type data: (n,d)-sized Numpy array of floats
	:type query: (d)-sized Numpy array of floats

	:result: The distances of the data vectors to the query.
	:rtype: (n)-sized Numpy array of floats
	"""
	norm_function = norms[norm]
	distances = np.zeros((len(data),), dtype=np.float32)
	for i, d in enumerate(data):
		distances[i] = norm_function(d-query)
	return distances

def compute_distance(gallery_pictures, probe_pictures, norm='L2'):
    total_array_distance = np.zeros(dtype=object, shape=200)
    for i, pic, in enumerate(probe_pictures):
        total_array_q = np.zeros(dtype=object, shape=273)
        for j, n in enumerate(gallery_pictures):
            total_array_q[j] = compute_distances_individual(n,pic)
            total_array_distance[i] = total_array_q
    return total_array_distance

def radius_search_bruteforce(data, q, radius, norm = 'L2'):
    distance_matrix = compute_distance(data, q, norm)
    res_all_probe = np.zeros(dtype=object, shape=200)
    for i, row in enumerate(distance_matrix):
        id_list = []
        for j, second_row in enumerate(row):
            for k, element in enumerate(second_row):
                if element <= radius:
                    id_list.append((j,k,element))
        res_all_probe[i] = id_list
    return res_all_probe

from math import sqrt

def radius_opti(data):
    mean_total_list = []
    for i, row in enumerate(data):
        mean_row_list = []
        for j, element in enumerate(row):
            tmp = row
            mean_row_list.append(np.mean(compute_distances_individual(np.delete(tmp, j, axis=0), element)))
        mean_row = np.mean(mean_row_list)
        mean_total_list.append(mean_row)
    mean_total = np.mean(mean_total_list)#moyenne de distance entre deux photos d'une meme pers
    std_total = sqrt(np.var(mean_total_list))#ecart type
    return mean_total + std_total #radius opti


In [3]:
data_generator = GalleryProbesGenerator(path_to_dataset='data/dataset1/', probes_length=200)

gallery_names, gallery_pictures = data_generator.get_gallery()
probe_names, probe_pictures = data_generator.get_merged_probes()
ground_truth = data_generator.get_ground_truth()

# radius = radius_opti(gallery_pictures)
radius = 1970000

In [4]:
def authenticate(probe_image, gallery_images):
    search_among_gallery = radius_search_bruteforce(data=gallery_images, q=probe_image, radius=radius)
    result = (False, None)
    if len(search_among_gallery)>0: # At least one neighbor found -> authentication accepted
        id_neighbors_found = list()
        for element in search_among_gallery:
            if element[0] not in id_neighbors_found:
                id_neighbors_found.append(element[0])
        result = (True, np.array(id_neighbors_found))
    return result

In [5]:
def authenticate_all_probes(gallery_images, probes_images):
    results_tests_probes = list()
    for probe_image in probes_images:
        results_tests_probes.append(authenticate(probe_image, gallery_images))
    return np.array(results_tests_probes)

In [ ]:
result_probes_authentication = authenticate_all_probes(gallery_pictures, probe_pictures)

def compute_metrics(result_probes_authentication, ground_truth):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i, probe_authentication in enumerate(result_probes_authentication):
        #True positive : access authorized for a registered person
        if probe_authentication[0] == True and ground_truth[i][0] in probe_authentication[1]:
            TP += 1

        #False positive : access authorized for an unregistered person
        if probe_authentication[0] == True and ground_truth[i][0] not in probe_authentication[1]:
            FP += 1

        #True negative : access denied for an unregistered person
        if probe_authentication[0] == False and ground_truth[i][0] == False:
            TN += 1

        #False negative : access denied for a registered person
        if probe_authentication[0] == False and ground_truth[i][0] == True:
            FN +=1

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    specificity = TN / (TN + FP)

    return accuracy, precision, recall, specificity